In [179]:
import numpy as np

### Game Class

In [180]:
class PuzzleState:

    def __init__(self, puzzle, car_fuels):
        self.puzzle = puzzle
        self.car_fuels = car_fuels

    @classmethod
    def construct_initial_puzzlestate(cls, puzzle_configuration_line):
        return cls(cls.parse_initial_puzzle(puzzle_configuration_line[0]), cls.parse_initial_car_fuels(puzzle_configuration_line))

    @classmethod
    def construct_puzzlestate_copy(cls, puzzlestate):
        return cls(np.copy(puzzlestate.puzzle), puzzlestate.car_fuels.copy())

    @staticmethod
    def parse_initial_puzzle(puzzle_configuration):
        array_line_input = np.array(list(puzzle_configuration))
        initial_puzzle = np.reshape(array_line_input, (6, 6))
        return initial_puzzle

    @staticmethod
    def parse_initial_car_fuels(puzzle_configuration_line):
        initial_car_fuels = dict.fromkeys(puzzle_configuration_line[0].replace('.', '')[::-1])
        for predefined_car_fuel in puzzle_configuration_line[1:]:
            initial_car_fuels[predefined_car_fuel[0]] = int(predefined_car_fuel[1])
        for car, fuel in initial_car_fuels.items():
            if fuel is None:
                initial_car_fuels[car] = 100
        return initial_car_fuels

    # TODO implement this
    def deparse_puzzle_configuration(self):
        return None

    def is_equal_state(self, puzzle):
        return np.array_equal(self.puzzle, puzzle)

    def is_goal_state(self):
        return np.all(self.puzzle[2, np.array([4,5])] == 'A')

    def get_children_puzzlestates(self):
        children_nodes = []
        print('all cars for ref:', self.car_fuels)
        print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

        self.if_car_at_valet()  # if car at exit then removes it

        for car in [car for car in self.car_fuels if self.car_fuels[car] > 0]:  # only iterate over cars with fuel

            print('puzzle at start of car iter\n', self.puzzle)

            print('car name:', car)
            car_indices = list(zip(*np.where(self.puzzle == car)))  # np.where() returns tuple (rows, cols) so zip to get coords
            if not car_indices: # in case the car was removed by valet
                continue
            print('car indices:', car_indices)
            car_length_orientation_vector = np.sum(np.diff(car_indices, axis=0), axis=0)  # 2-in-1 to get diff b/w pairs for orientation and sum for length-1
            print(car_length_orientation_vector)

            current_axis_identifier, car_axis = self.get_puzzle_car_axis(car_length_orientation_vector, car_indices)

            print('car axis:', car_axis)
            empty_spot_indices = np.where(car_axis == '.')[0]
            car_indices_one_axis = list(zip(*car_indices))[current_axis_identifier ^ 1]  # zip back for axes and index correct tuple
            possible_movements_temp = np.sort(np.concatenate((np.array(car_indices_one_axis), empty_spot_indices)))  # periods and the car indices
            # print('periods and the car', possible_movements_temp)
            # print('car', car_indices_one_axis)
            car_indices_in_possible_movements_temp = np.searchsorted(possible_movements_temp, car_indices_one_axis)
            # print('car index in "periods and the car"', car_indices_in_possible_movements_temp)
            possible_movements_temp_diff = np.ediff1d(possible_movements_temp)
            # print('diff bw poss movements', temp_possible_movements_diff)

            displacement_indices_direction_tuples = []  # tuple (displacement, slice indices in axis, direction string)
            car_length = np.sum(car_length_orientation_vector) + 1
            min_car_index_one_axis =min(car_indices_one_axis)
            max_car_index_one_axis = max(car_indices_one_axis)

            # left or up displacement
            min_car_index_in_possible_movements_temp = min(car_indices_in_possible_movements_temp)
            while min_car_index_in_possible_movements_temp - 1 >= 0:
                if possible_movements_temp_diff[min_car_index_in_possible_movements_temp - 1] == 1:
                    # car min index can be moved to pos_mov_temp[min_car_ind_pos_mov_temp - 1]
                    displacement = possible_movements_temp[min_car_index_in_possible_movements_temp - 1] - min_car_index_one_axis
                    movement_direction_string = 'left' if current_axis_identifier == 0 else 'up'
                    car_axis_slice_indices = (max_car_index_one_axis - car_length + displacement + 1, max_car_index_one_axis + 1)
                    displacement_indices_direction_tuples.append((displacement, car_axis_slice_indices, movement_direction_string))
                    print('can move {} by {}'.format(movement_direction_string, displacement))
                    min_car_index_in_possible_movements_temp -= 1
                else:
                    break

            # right or down displacement
            max_car_index_in_possible_movements_temp = max(car_indices_in_possible_movements_temp)
            while max_car_index_in_possible_movements_temp < np.size(possible_movements_temp_diff):
                if possible_movements_temp_diff[max_car_index_in_possible_movements_temp] == 1:
                    # car max index can be moved to pos_mov_temp[max_car_ind_pos_mov_temp + 1]
                    displacement = possible_movements_temp[max_car_index_in_possible_movements_temp + 1] - max_car_index_one_axis
                    movement_direction_string = 'right' if current_axis_identifier == 0 else 'down'
                    car_axis_slice_indices = (min_car_index_one_axis, min_car_index_one_axis + car_length + displacement)
                    displacement_indices_direction_tuples.append((displacement, car_axis_slice_indices, movement_direction_string))
                    print('can move {} by {}'.format(movement_direction_string, displacement))
                    max_car_index_in_possible_movements_temp += 1
                else:
                    break

            for movement_tuple in displacement_indices_direction_tuples:
                print('inside movement iter')

                if self.car_fuels[car] >= abs(movement_tuple[0]):  # only complete move if car has enough fuel
                    puzzlestate_copy = PuzzleState.construct_puzzlestate_copy(self)
                    print('car fuels before move made', self.car_fuels)
                    puzzlestate_copy.car_fuels[car] -= abs(movement_tuple[0])
                    _, car_axis = puzzlestate_copy.get_puzzle_car_axis(car_length_orientation_vector, car_indices)
                    print('puzzle before move made \n', puzzlestate_copy.puzzle)
                    print('car axis before move made \n', car_axis)
                    car_axis_slice = car_axis[movement_tuple[1][0]:movement_tuple[1][1]]
                    car_axis_slice[:] = np.roll(car_axis_slice, movement_tuple[0])
                    print('car axis after move made \n', car_axis)
                    print('puzzle after move made \n', puzzlestate_copy.puzzle)
                    print('car fuels after move made', puzzlestate_copy.car_fuels)
                    print('self.puzzle after move made\n', self.puzzle)
                    movement_tuple_formatted = (car, movement_tuple[2], movement_tuple[0])
                    children_nodes.append((puzzlestate_copy, movement_tuple_formatted))

            print('\n\n')
        return children_nodes

    def get_puzzle_car_axis(self, car_length_orientation_vector, car_indices):
        current_axis_identifier = None
        car_axis = None
        if car_length_orientation_vector[0] == 0 and car_length_orientation_vector[1] > 0:  # horizontal -> bit 0
            current_axis_identifier = 0
            car_axis = self.puzzle[car_indices[0][current_axis_identifier], :]
        elif car_length_orientation_vector[0] > 0 and car_length_orientation_vector[1] == 0:  # vertical -> bit 1
            current_axis_identifier = 1
            car_axis = self.puzzle[:, car_indices[0][current_axis_identifier]]
        return current_axis_identifier, car_axis

    def if_car_at_valet(self):
        neighbour_pairs_indices = np.where(self.puzzle[2, :-1] == self.puzzle[2, 1:])[0]  # compares equality of neighbour pairs
        exit_car_indices = -1
        for i in range(4, -1, -1):
            possible_exit_car_indices = np.arange(i, 5)
            if np.all(np.in1d(possible_exit_car_indices, neighbour_pairs_indices)):
                exit_car_indices = np.append(possible_exit_car_indices, 5)
            else:
                break
        if exit_car_indices != -1:
            self.puzzle[2, exit_car_indices[0]:] = '.'

### State Space Search Class

In [181]:
class Node:

    def __init__(self, puzzle_state, parent, children):
        self.puzzle_state = puzzle_state
        self.parent = parent
        self.children = children
        self.g = 0
        self.h = 0
        self.f = 0

    def increment_g(self):
        self.g = + 1

    def set_h(self, h):
        self.h = h

    def update_f(self):
        self.f = self.g + self.h

### Load Input File

In [182]:
PATH_TO_INPUT_FILE = '../Sample/sample-input.txt'

def parse_input_file(path):
    puzzlestates_temp = []
    with open(path, 'r') as f:
        lines_nonempty = filter(None, (line.rstrip() for line in f))
        for line in [line for line in lines_nonempty if line[0] != '#']:
            puzzle_configuration_line = line.split(' ')
            puzzle_state = PuzzleState.construct_initial_puzzlestate(puzzle_configuration_line)
            puzzlestates_temp.append(puzzle_state)
    return puzzlestates_temp

puzzlestates = parse_input_file(PATH_TO_INPUT_FILE)
for puzzlestate in puzzlestates:
    puzzlestate.get_children_puzzlestates()

all cars for ref: {'L': 100, 'F': 100, 'H': 100, 'G': 100, 'K': 100, 'M': 100, 'D': 100, 'A': 100, 'I': 100, 'C': 100, 'J': 100, 'B': 100}
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
puzzle at start of car iter
 [['B' 'B' 'I' 'J' '.' '.']
 ['.' '.' 'I' 'J' 'C' 'C']
 ['.' '.' 'I' 'A' 'A' 'M']
 ['G' 'D' 'D' 'K' '.' 'M']
 ['G' 'H' '.' 'K' 'L' '.']
 ['G' 'H' 'F' 'F' 'L' '.']]
car name: L
car indices: [(4, 4), (5, 4)]
[1 0]
car axis: ['.' 'C' 'A' '.' 'L' 'L']
can move up by -1
inside movement iter
car fuels before move made {'L': 100, 'F': 100, 'H': 100, 'G': 100, 'K': 100, 'M': 100, 'D': 100, 'A': 100, 'I': 100, 'C': 100, 'J': 100, 'B': 100}
puzzle before move made 
 [['B' 'B' 'I' 'J' '.' '.']
 ['.' '.' 'I' 'J' 'C' 'C']
 ['.' '.' 'I' 'A' 'A' 'M']
 ['G' 'D' 'D' 'K' '.' 'M']
 ['G' 'H' '.' 'K' 'L' '.']
 ['G' 'H' 'F' 'F' 'L' '.']]
car axis before move made 
 ['.' 'C' 'A' '.' 'L' 'L']
car axis after move made 
 ['.' 'C' 'A' 'L' 'L' '.']
puzzle after move made 
 [['B' 'B' 'I' 'J' '.' '.']
 ['.' '.' 'I' 'J'

In [183]:
# [['B' 'B' 'I' 'J' '.' '.']
#  ['.' '.' 'I' 'J' 'C' 'C']
#  ['.' '.' 'I' 'A' 'A' 'M']
#  ['G' 'D' 'D' 'K' '.' 'M']
#  ['G' 'H' '.' 'K' 'L' '.']
#  ['G' 'H' 'F' 'F' 'L' '.']]

In [184]:
# puzzle = puzzles[0].puzzle
# print(puzzle)
# print("\n")
# # index a row
# print(puzzle[0])
# # index a column but show as horizontal reg array
# print(puzzle[:, 5])
# # index a column but show as a vertical column array of 1element arrays
# print(puzzle[:, np.array([5])])
# # location for ambulance to win
# print(puzzle[2, np.array([4,5])])

### Heuristic Functions

In [186]:
def h1(puzzle):
    index = np.ndarray.flatten(np.argwhere(puzzle.puzzle[2] == 'A'))
    test = puzzle.puzzle[2, index[-1]+1:]
    test = test[test != '.']
    return len(set(test))

# Other solution
# for arr in puzzles:
#     print(arr.puzzle[2])
#     _, idx = np.unique(arr.puzzle[2], return_index=True)
#     temp = arr.puzzle[2][np.sort(idx)]
#     arr_no_duplicates = temp[temp != '.']
#     print((arr_no_duplicates.size - 1) - (np.where(arr_no_duplicates == 'A')[0][0]))

In [187]:
def h2(puzzle):
    index = np.ndarray.flatten(np.argwhere(puzzle.puzzle[2] == 'A'))
    test = puzzle.puzzle[2, index[-1]+1:]
    test = test[test != '.']
    return len(test)

In [188]:
def h3(puzzle):
    return h1(puzzle) * 4

In [189]:
def board(puzzle):
    for element in puzzle:
        print(*element)